# Download weather data for Prague

or any other city

In [1]:
from datetime import datetime

import polars as pl
from meteostat import Hourly, Stations

In [2]:
CITY = "Prague"
COUNTRY = "Czechia"

In [3]:
# Load our cities dataset and find the city coordinates
cities = pl.read_parquet("../data/worldcities.parquet")
city_info = cities.filter(city=CITY, country=COUNTRY).row(0, named=True)
city_info


{'city': 'Prague',
 'city_ascii': 'Prague',
 'lat': 50.0875,
 'lng': 14.4214,
 'country': 'Czechia',
 'iso2': 'CZ',
 'iso3': 'CZE',
 'admin_name': 'Praha',
 'capital': 'primary',
 'population': 1357326,
 'id': 1203744823}

In [4]:
# Find nearby weather stations
s = Stations()
s = s.nearby(
    lat=city_info["lat"],
    lon=city_info["lng"],
    radius=100000  # in m?
)
stations = s.fetch(10)
stations   # pandas DataFrame

,name,country,region,wmo,icao,latitude,longitude,elevation,timezone,hourly_start,hourly_end,daily_start,daily_end,monthly_start,monthly_end,distance
id,,,,,,,,,,,,,,,,
11520,Praha-Libus,CZ,PR,11520,<NA>,50.0167,14.4500,304.0,Europe/Prague,NaT,NaT,1973-01-01,2024-06-12,1972-01-01,2021-01-01,8133.107961
11567,Praha Kbely,CZ,PR,11567,<NA>,50.1167,14.5333,286.0,Europe/Prague,2022-04-19,2022-04-23,2022-04-20,2022-04-22,NaT,NaT,8616.206889
11518,Praha / Ruzyne,CZ,ST,11518,LKPR,50.1000,14.2500,365.0,Europe/Prague,1929-08-02,2024-06-21,1940-01-01,2024-06-12,1940-01-01,2022-01-01,12305.608643
LKVO0,Vodochody,CZ,ST,<NA>,LKVO,50.2166,14.3958,280.0,Europe/Prague,2020-01-14,2024-06-20,NaT,NaT,NaT,NaT,14470.675695
11509,Doksany,CZ,US,11509,<NA>,50.4667,14.1667,158.0,Europe/Prague,NaT,NaT,NaT,NaT,NaT,NaT,45885.523266
11464,Milesovka,CZ,UK,11464,<NA>,50.5500,13.9333,836.0,Europe/Prague,NaT,NaT,1906-05-01,2024-06-12,1906-01-01,2021-01-01,62013.961753
11624,Caslav,CZ,ST,11624,LKCV,49.9333,15.3833,242.0,Europe/Prague,1997-01-01,2024-06-21,1997-03-09,2022-04-25,2005-01-01,2022-01-01,70842.378076
11502,Usti Nad Labem,CZ,US,11502,<NA>,50.6833,14.0333,377.0,Europe/Prague,NaT,NaT,NaT,NaT,NaT,NaT,71736.776174
11628,Kramolin-Kosetice,CZ,VY,11628,<NA>,49.5833,15.0833,534.0,Europe/Prague,NaT,NaT,NaT,NaT,NaT,NaT,73461.700845


In [7]:
# Take the third one (Praha/Ruzyně) for Prague
station_id = stations.index[2]
station_id

'11518'

In [8]:
# Get all accessible data
station = stations.loc[station_id]
source = Hourly(station_id, start=station["hourly_start"], end=datetime.now())
source

In [9]:
# Create a DataFrame from it (we are learning polars, aren't we?)
df = pl.from_pandas(source.fetch().reset_index(), nan_to_null=True)
# Be explicit about the Nones
df = df.fill_nan(None)
df

time,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco
datetime[ns],f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1929-08-02 06:00:00,17.2,null,null,null,null,270.0,16.6,null,1010.2,null,null
1929-08-02 18:00:00,17.8,null,null,null,null,270.0,16.6,null,1012.4,null,null
1929-08-03 06:00:00,15.0,null,null,null,null,20.0,3.6,null,1019.6,null,null
1929-08-04 06:00:00,15.0,null,null,null,null,0.0,0.0,null,1018.4,null,null
1929-08-05 06:00:00,17.2,null,null,null,null,180.0,3.6,null,1012.0,null,null
…,…,…,…,…,…,…,…,…,…,…,…
2024-06-22 03:00:00,17.0,15.7,92.0,0.3,null,240.0,20.4,33.3,1011.1,null,18.0
2024-06-22 04:00:00,16.8,15.5,92.0,0.2,null,241.0,20.4,33.3,1011.3,null,7.0
2024-06-22 05:00:00,16.7,15.2,91.0,1.2,null,242.0,20.4,33.3,1011.7,null,7.0


In [10]:
# Save
df.write_parquet(f"../data/{CITY.lower()}-meteostat.parquet")